In [6]:
import sys
sys.path.append('..')

In [7]:
import numpy as np
from rblr import IFB, ClassicalBootstrap, Preprocessor, \
    StratifiedBootstrap, ModifiedStraitifiedBootstrap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline

## 1. Load and clean dataset
### 1.1 load dataset
The dataset is the breast cancer dataset from 
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29 It contains 569 observations and each 
observation has 32 features. There are two types of labels. "B" stands for benign, "M" stands for malignant. 
31 out of 32 features consist of numeric values, while the last feature contains a large number of nan values, and the 
first feature is the ID number.

In [9]:
df = pd.read_csv('./test_dataset/breast_cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### 1.2 clean data and encode labels into numeric values
Drop the two redundant columns. Encode "B" into 0 and "M" into 1.

In [10]:
# remove redundant columns
df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)
# encode diagnosis
df['diagnosis'] = df['diagnosis'].map({'B': 0, 'M': 1})

### 1.3 prepare data for train and test
* extract X and y
* min-max normalize each feature of X according to $$x^{'} = \frac{x - min(x)}{max(x) - min(x)}$$
so that values of each feature are located in range from 0 to 1.
* split the dataset into train and test datasets, where 0.2 proportion of the dataset will be used as test dataset.

In [11]:
# split X and y
X = df.drop(['diagnosis'], axis=1)
y = df['diagnosis']
print(y.value_counts())

# min-max scale
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

0    357
1    212
Name: diagnosis, dtype: int64


### 1.4 define a contaminating function to generate outliers
Given a fraction  $\lambda = N_o/N_{train} $
 (default 0.2), representing the fraction of outliers in clean training dataset, and a factor  $f$
  representing the fraction of outliers that will be labelled with 1, that is, the outliers lying on the the same side 
  as the majority of class 0. The outliers are generated as follows:
1. calculate number of outliers : $ N_o = N_{train}\cdot \lambda $
 , and number of outliers that will be labelled with 1:  $N^{(0)}_o= N_o\cdot f$
 , and the number of outliers that will be labelled with 0:  $N^{(1)}_o = N_o − N^{(0)}_o$
 .
 
2. Split the training data into two classes accoring to the label, and randomly sample from each class  $N^{(0)}_o$,
$N^{(1)}_o$ number of outliers respectively. We obtain two sampled datasets  $X^{(0)}_s$, $X^{(1)}_s$.
3. for each observation $\mathbf{x}$ in the sampled datasets, generate an oulier as follows, take an example of class 0:
$$\mathbf{x}_o= t_{scale}\cdot \mathbf{x} + \mathbf{x}_{noise}$$ where $t_{scale}$ (default 10) is a magnifer factor 
and $\mathbf{x}_{noise}$ is the gaussian noise. Then, reverse the label, i.e. label the outlier with 1.
4. concatenate the  $\mathbf{X}$ and label matrix with training data


In [16]:
# define a contamination function
def contaminate(X_train, y_train, contamination=0.2, scale=10, label_percentage=0.5):
    if ((np.max(X_train, axis=0) - np.min(X_train, axis=0)) >1.1).any():
        print(np.max(X_train, axis=0) - np.min(X_train, axis=0))
        raise ValueError("the input matrix is not min-max scaled")
    if label_percentage < 0 or label_percentage >1:
        raise ValueError("label percentage can only be between 0 and 1")
    np.random.seed()
    n_out = int(X_train.shape[0] * contamination)
    n_out_0 = int(n_out * label_percentage)
    n_out_1 = n_out - n_out_0
    
    # generate outliers
    X_cat_0 = X_train[y_train==0]
    X_out_0 = scale * X_cat_0[np.random.choice(X_cat_0.shape[0], n_out_0)] + \
              0.1 * scale * np.random.randn(n_out_0, X_cat_0.shape[1])
    # reverse label of 0-class into 1
    y_out_0 = np.ones(n_out_0, dtype=int)
    
    X_cat_1 = X_train[y_train==1]
    X_out_1 = scale * X_cat_1[np.random.choice(X_cat_1.shape[0], n_out_1)] + \
              0.1 * scale * np.random.rand(n_out_1, X_cat_1.shape[1])
    # reverse label of 1-class into 0
    y_out_1 = np.zeros(n_out_1, dtype=int)
    
    # concatenate X and y
    X_train = np.concatenate((X_train, X_out_0, X_out_1), axis=0)
    y_train = np.concatenate((y_train, y_out_0, y_out_1), axis=0)
    return X_train, y_train

In [17]:
contamination = 0.3
scale = 1
X_train_conta, y_train_conta = contaminate(X_train, y_train, contamination, scale=scale)

## 2. Classification with various models and methods
Use different models and methods to fit the clean and contaminated data with contamination factor 0.3 respectively, 
then predict the test dataset and evaluate the accuracies of the models.
### 2.1 Classical Logisitc regression

In [18]:
# classical LR on clean data
classical_lr = LogisticRegression(solver='lbfgs', max_iter=500)
classical_lr.fit(X_train, y_train)
print('classical LR on clean data, accuracy: %.2f' % (classical_lr.score(X_test, y_test)))

classical LR on clean data, accuracy: 0.97


In [19]:
# classical LR on contaminated data
classical_lr.fit(X_train_conta, y_train_conta)
print("classical LR on %.2f contaminated data, accuracy: %.2f" 
      %(contamination, classical_lr.score(X_test, y_test)))

classical LR on 0.30 contaminated data, accuracy: 0.85


### 2.2 Classical Bootstrap
# classical bootstrap on clean data

In [20]:
classical_boot = ClassicalBootstrap(max_iter=500)
classical_boot.fit(X_train, y_train)
print('classical Bootstrap on clean data, accuracy: %.2f' % (classical_boot.score(X_test, y_test)))


classical Bootstrap on clean data, accuracy: 0.88


In [21]:
# classical bootstrap on contaminated data
classical_boot.fit(X_train_conta, y_train_conta)
print('classical bootstrap on %.2f contaminated data, accuracy: %.2f' 
      %(contamination, classical_boot.score(X_test, y_test)))

classical bootstrap on 0.30 contaminated data, accuracy: 0.91


### 2.3 Classical Influence Function Bootstrap

In [22]:
# classical IFB method on clean data
ifb = IFB(fit_intercept=True)
ifb.fit(X_train, y_train, quantile_factor=0.9, max_iter=500)
print('classical IFB on clean data, accuracy: %.2f' %(ifb.score(X_test, y_test)))

classical IFB on clean data, accuracy: 0.96


In [23]:
# classical IFB method on contaminated data
ifb.fit(X_train_conta, y_train_conta, quantile_factor=0.9, max_iter=500)
print('classical IFB on %.2f contaminated data, accuracy: %.2f' 
      %(contamination, ifb.score(X_test, y_test)))

classical IFB on 0.30 contaminated data, accuracy: 0.86


### 2.4 Preprocessed Influence Function Bootstrap

In [24]:
# preprocessed IFB on clean data
preprocessor = Preprocessor()
X_train_prep , y_train_prep = preprocessor.fit_transform(X_train, y_train)
ifb.fit(X_train_prep, y_train_prep)
print("preprocessed IFB on clean data, accuracy: %.2f" %(ifb.score(X_test, y_test)))

preprocessed IFB on clean data, accuracy: 0.96


In [25]:
# preprocessed IFB on contaminated data
preprocessor = Preprocessor()
X_train_conta_prep, y_train_conta_prep = preprocessor.fit_transform(X_train_conta, y_train_conta)
ifb.fit(X_train_conta_prep, y_train_conta_prep)
print('preprocessed IFB on %.2f contaminated data, accuracy: %.2f' 
      %(contamination, ifb.score(X_test, y_test)))

preprocessed IFB on 0.30 contaminated data, accuracy: 0.87


### 2.5 Stratified Bootstrap
* Number of strata: 5

In [26]:
# stratified bootstrap on clean data
stratified_boot = StratifiedBootstrap()
stratified_boot.fit(X_train, y_train, n_strata=5, metric='residual')
print('stratified bootstrap on clean data, accuracy: %.2f'
      % (stratified_boot.score(X_test, y_test)))

stratified bootstrap on clean data, accuracy: 0.89


* Number of bootstrap samples: 10
* Number of strata: 5
* **Note**: here 10 parallelisms are used. The time effectiveness is somehow unstable, depending on the dataset and bootstrap 
samples. In good cases the perfomance can reach 147 seconds.

In [30]:
# stratified bootstrap on contaminated data
t1 = time.time()
stratified_boot = StratifiedBootstrap(warm_start=True)
stratified_boot.fit(X_train_conta, y_train_conta,
                    n_bootstrap=5, n_strata=3, metric='residual',
                    verbose=True, n_jobs=None, )
print('stratified boostrap on %.2f contaminated data, accuracy: %.2f'
      %(contamination, stratified_boot.score(X_test, y_test)))
print("consumed time: %.2f s" % (time.time() - t1))

-----fit bootstrap sample No.0-----
-----fit bootstrap sample No.1-----
-----fit bootstrap sample No.2-----
-----fit bootstrap sample No.3-----
-----fit bootstrap sample No.4-----
L-BFGS failed to converge: 0 / 5 times
stratified boostrap on 0.30 contaminated data, accuracy: 0.95
consumed time: 219.55 s


### 2.6 Modified Stratified Bootstrap
* Number of bootstrap samples: 20
* Number of strata: 2

In [31]:
# modified stratified bootstrap on clean data
modified_strat_boot = ModifiedStraitifiedBootstrap()
modified_strat_boot.fit(X_train, y_train, n_bootstrap=20, n_strata=2)
print('modified stratified bootstrap on clean data, accuracy: %.2f'
      %(modified_strat_boot.score(X_test, y_test)))

modified stratified bootstrap on clean data, accuracy: 0.85


On contaminated data:
* Number of bootstrap samples: 20
* Number of strata: 3

In [32]:
# modified stratified bootstrap on contaminated data
modified_strat_boot = ModifiedStraitifiedBootstrap(max_iter=500)
modified_strat_boot.fit(X_train_conta, y_train_conta, n_bootstrap=20, n_strata=3)
print('modified stratified bootstrap on %.2f contaminated data, accuracy: %.2f'
      %(contamination, modified_strat_boot.score(X_test, y_test)))

modified stratified bootstrap on 0.30 contaminated data, accuracy: 0.88


## 3. Summary
The simulation results are summarized as follows:

|Methods| clean data | 0.3 contamination |
|---|:---:|:---:|
|Classical LR | 0.96 | 0.63 |
|Classical Bootstrap | 0.94 | 0.39 |
|Classical IFB | 0.97 | 0.80 |
|PreprocessedIFB | 0.97 | 0.97 |
|Stratified Bootstrap | 0.92 | 0.94 |
|Modified Stratified Bootstrap | 0.94 | 0.94 |


